Mounting Google Drive to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

1.Importing necessary modules

2.Defining function to determine IoU

In [ ]:
import os
import cv2

def bbox_iou(box1, box2):
	
    b1_x1, b1_x2 = (box1[0] - box1[2] / 2), (box1[0] + box1[2] / 2)
    b1_y1, b1_y2 = (box1[1] - box1[3] / 2), (box1[1] + box1[3] / 2)
    b2_x1, b2_x2 = (box2[0] - box2[2] / 2), (box2[0] + box2[2] / 2)
    b2_y1, b2_y2 = (box2[1] - box2[3] / 2), (box2[1] + box2[3] / 2)

    # Intersection area
    inter = (min(b1_x2, b2_x2) - max(b1_x1, b2_x1))*(min(b1_y2, b2_y2) - max(b1_y1, b2_y1))

    # Union Area
    w1, h1 = b1_x2 - b1_x1, b1_y2 - b1_y1
    w2, h2 = b2_x2 - b2_x1, b2_y2 - b2_y1
    union = (w1 * h1 + 1e-16) + w2 * h2 - inter
    iou = inter / union 

    return iou

Requirements :

1. All images and labels in path_output and path_true should not be present in sub-directories of the given path and all of them should be in the same given path.

2. Create directories with paths as path_display,path_fp,path_tn for outputs to be saved.


Execute the following cell for plotting ground truth bounding boxes and predicted bounding boxes on the images

In [ ]:
path_output  = #<Path to the bbox output given by network>
path_true    = #<Path to original image and true bbox directory> 
path_display = #<Path where images are to be stored> 
list_img = []

for r, d, f in os.walk(path_true):
    for file in f:
        if '.jpg' in file:
          	list_img.append(file)

for img_loc in list_img:
    loc = img_loc[:-4] + '.txt'
    file_output = path_output + loc
    file_true = path_true + loc
    img_path = path_true + img_loc
    img_path_out = path_display + img_loc
    bbox_output = []
    bbox_true = []
    line_1 = []
    line_2 = []
    try:
        line_1 = open(file_output,"r").readlines()
    except FileNotFoundError:
        print("Wrong file or file path")
    try:
        line_2 = open(file_true,"r").readlines()
    except FileNotFoundError:
        print("Wrong file or file path")    
 
    for entry in line_1:
        bbox_1 = (entry.split(' ')[1:5])
        bbox_1 = [float(item) for item in bbox_1]
        bbox_output.append(bbox_1)
    for entry in line_2:
        bbox_2 = (entry.split(' ')[1:5])
        bbox_2 = [float(item) for item in bbox_2]
        bbox_true.append(bbox_2)  
   
    print(img_path)
    frame = cv2.imread(img_path)
    height,width = frame.shape[0],frame.shape[1]
    for (x,y,w,h) in bbox_output:
        left =int((x - (w/2))*width)
        right = int((x + (w/2))*width)
        bottom = int((y + (h/2))*height)
        top = int((y - (h/2))*height)
        cv2.rectangle(frame, (left, top), (right, bottom), (0,255, 0), 2)
    for [x,y,w,h] in bbox_true:
        left =int((x - (w/2))*width)
        right = int((x + (w/2))*width)
        bottom = int((y + (h/2))*height)
        top = int((y - (h/2))*height)
        cv2.rectangle(frame, (left, top), (right, bottom), (0,0,255),3)
    cv2.imwrite(img_path_out,frame)
    
cv2.destroyAllWindows()

Execute the following cell for plotting only false-positive datapoints in the given test set assuming IoU for such a false-positive with all bounding boxes in the annotated output is less than 0.5

False positive - Present in output not in dataset

In [ ]:
path_output  = #<Path to the bbox output given by network>
path_true    = #<Path to original image and true bbox directory> 
path_fp      = #<Path where images are to be stored> 
list_output = []

for r, d, f in os.walk(path_output):
    for file in f:
        if '.txt' in file:
            if file != 'classes.txt':
          	    list_output.append(file)

for loc in list_output:
    file_output = path_output + loc
    file_true = path_true + loc
    img_loc = loc[:-4] + '.jpg'
    img_path = path_true + img_loc
    img_path_out = path_fp + img_loc
    bbox_output = []
    bbox_true = []
    line_1 = []
    line_2 = []
    fp_bbox = []
    line_1 = open(file_output,"r").readlines()
    try:
        line_2 = open(file_true,"r").readlines()
    except FileNotFoundError:
        print("Wrong file or file path")
    if line_2 == [] : 
        for entry in line_1:
            bbox_1 = (entry.split(' ')[1:5])
            fp_bbox.append(bbox_1)
    else:
        for entry in line_1:
            bbox_1 = (entry.split(' ')[1:5])
            bbox_1 = [float(item) for item in bbox_1]
            bbox_output.append(bbox_1)
        for entry in line_2:
            bbox_2 = (entry.split(' ')[1:5])
            bbox_2 = [float(item) for item in bbox_2]
            bbox_true.append(bbox_2)  
        for i in bbox_output:
            iou = []
            for j in bbox_true:
                iou.append(bbox_iou(i,j))
            if max(iou) > 0.5:
                pass
            else : 
                fp_bbox.append([str(var) for var in i])
    if fp_bbox == [] :
        pass
    else:
        fp_img = []
        for entry in fp_bbox:
            bbox_fp = [float(item) for item in entry]
            fp_img.append(bbox_fp)
  
        frame = cv2.imread(img_path)
        height,width = frame.shape[0],frame.shape[1]
        for (x,y,w,h) in fp_img:
            left =int((x - (w/2))*width)
            right = int((x + (w/2))*width)
            bottom = int((y + (h/2))*height)
            top = int((y - (h/2))*height)
            cv2.rectangle(frame, (left, top), (right, bottom), (0,255, 0), 2)
        cv2.imwrite(img_path_out,frame)

cv2.destroyAllWindows()

Execute the following cell for plotting only false-negative datapoints in the given test set assuming IoU for such a false-negative with all bounding boxes in the predicted output is less than 0.5

False-negative - Present in ground truth not present in output

In [ ]:
path_output  = #<Path to the bbox output given by network>
path_true    = #<Path to original image and true bbox directory> 
path_fn      = #<Path where images are to be stored> 
list_true = []

for r, d, f in os.walk(path_true):
    for file in f:
        if '.txt' in file:
            if file != 'classes.txt':
          	    list_true.append(file)

for loc in list_true:
    file_output = path_output + loc
    file_true = path_true + loc
    img_loc = loc[:-4] + '.jpg'
    img_path = path_true + img_loc
    img_path_out = path_fn + img_loc
    bbox_output = []
    bbox_true = []
    line_1 = []
    line_2 = []
    fn_bbox = []
    line_1 = open(file_true,"r").readlines()
    try:
        line_2 = open(file_output,"r").readlines()
    except FileNotFoundError:
        print("Wrong file or file path")
    if line_2 == [] : 
        for entry in line_1:
            bbox_1 = (entry.split(' ')[1:5])
            fn_bbox.append(bbox_1)
    else:
        for entry in line_1:
            bbox_1 = (entry.split(' ')[1:5])
            bbox_1 = [float(item) for item in bbox_1]
            bbox_output.append(bbox_1)
        for entry in line_2:
            bbox_2 = (entry.split(' ')[1:5])
            bbox_2 = [float(item) for item in bbox_2]
            bbox_true.append(bbox_2)  
        for i in bbox_output:
            iou = []
            for j in bbox_true:
                iou.append(bbox_iou(i,j))
            if max(iou) > 0.5:
                pass
            else : 
                fn_bbox.append([str(var) for var in i])
    if fn_bbox == [] :
        pass
    else:
        fn_img = []
        for entry in fn_bbox:
            bbox_fn = [float(item) for item in entry]
            fn_img.append(bbox_fn)
  
        frame = cv2.imread(img_path)
        height,width = frame.shape[0],frame.shape[1]
        for (x,y,w,h) in fn_img:
            left =int((x - (w/2))*width)
            right = int((x + (w/2))*width)
            bottom = int((y + (h/2))*height)
            top = int((y - (h/2))*height)
            cv2.rectangle(frame, (left, top), (right, bottom), (0,0,255), 2)
        cv2.imwrite(img_path_out,frame)

   

cv2.destroyAllWindows()